In this code:
**Main Work:**
1. The workout number and durations for particular days for specific users are being calculated. The durations for individual workouts as well as for a day as a whole are being displayed in the 'final_populated_workouts.json' file.
2. The total number of calories burnt by individual workouts will be displayed against each workout aas well as the total calories burnt in one day will be displayed against the DAY value in the JSON file.

**Additional Tweaks**
1. An input function block has been defined in the start of the code that globalizes the user's input. An additional input of user's weight will be required by the database to calculate calories burnt. I have kept the data type of user weight an integer.
2. This code uses the combined logic of all the previous models like it was being done in the last file of "populateWeeklyPlansWithUserSpecificWorkouts.ipynb".

**Instrctions for Gujjar**
You probably know all this but:
1. Make sure the workouts csv dataset is loaded when you run this file
2. Kindly run it in the order it is given and keep storing the outputs in the same directories so that they can be worked on iteratively.


In [1]:
import pandas as pd
import json

In [2]:
#User input declaration and some conversion functions for user inputs
#It does not contain the conversion function for usr activity yet so the activity is still being taken as a textual Input of 'low','moderate' and 'high'

# Global user variables (initialized to None)
USER_AGE = None
USER_AGE_GROUP = None
USER_ACTIVITY_LEVEL = None
USER_REST_DAY = None
USER_PROGRAM_DURATION = None
USER_GOAL = None
USER_WEIGHT = None

def map_age_to_group(age):
    if age < 40:
        return 'adults'
    elif 40 <= age < 60:
        return 'middle_aged'
    else:
        return 'older_adult'

def load_user_profile():
    global USER_AGE, USER_AGE_GROUP, USER_ACTIVITY_LEVEL, USER_REST_DAY, USER_PROGRAM_DURATION, USER_GOAL

    #For now these values are being hardcoded
    USER_AGE = 45
    USER_ACTIVITY_LEVEL = 'high'  # 'low', 'moderate', 'high'
    USER_REST_DAY = 'Sunday'          # Any day like 'Monday', 'Wednesday', etc.
    USER_PROGRAM_DURATION = 60        # In any number of days
    USER_GOAL = 'weight_loss'        # 'weight_loss', 'stay_fit','gain_weight', 'build_muscle'
    USER_WEIGHT = 65  #in kgs

    # Map the age to age group because some parts of the model require a textual age group like adults, older_adults and middle_aged people
    USER_AGE_GROUP = map_age_to_group(USER_AGE)

    print({
        "Age": USER_AGE,
        "Age Group": USER_AGE_GROUP,
        "Activity Level": USER_ACTIVITY_LEVEL,
        "Preferred Rest Day": USER_REST_DAY,
        "Program Duration": USER_PROGRAM_DURATION,
        "Goal": USER_GOAL,
        "Weight": USER_WEIGHT
    })

#Printing
load_user_profile()


{'Age': 45, 'Age Group': 'middle_aged', 'Activity Level': 'high', 'Preferred Rest Day': 'Sunday', 'Program Duration': 60, 'Goal': 'weight_loss', 'Weight': 65}


In [3]:
#The only block with PYDATALOG you have already translated it into if-else conditions you may replace this block entirely with your translated code

!pip install pydatalog
from pyDatalog import pyDatalog

#Terms
pyDatalog.create_terms('Age, Activity, MetRange, Difficulty, Min, Max')

#Age group ranges
Min['adult'] = 15
Max['adult'] = 34
Min['middle_aged'] = 35
Max['middle_aged'] = 49
Min['older_adult'] = 50
Max['older_adult'] = 120

# MET range labels
pyDatalog.create_terms('recommended_met, recommended_difficulty')

#Lowercase difficulty terms to avoid inconsistency in data
beginner = 'beginner'
intermediate = 'intermediate'

#Recommended MET values and Difficulties for Adults(15-34)
recommended_met(Age, 'low', 'low') <= (Min['adult'] <= Age) & (Age <= Max['adult'])
recommended_met(Age, 'moderate', 'low') <= (Min['adult'] <= Age) & (Age <= Max['adult'])
recommended_met(Age, 'moderate', 'moderate') <= (Min['adult'] <= Age) & (Age <= Max['adult'])
recommended_met(Age, 'high', 'low') <= (Min['adult'] <= Age) & (Age <= Max['adult'])
recommended_met(Age, 'high', 'moderate') <= (Min['adult'] <= Age) & (Age <= Max['adult'])
recommended_met(Age, 'high', 'high') <= (Min['adult'] <= Age) & (Age <= Max['adult'])

recommended_difficulty(Age, 'low', beginner) <= (Min['adult'] <= Age) & (Age <= Max['adult'])
recommended_difficulty(Age, 'low', intermediate) <= (Min['adult'] <= Age) & (Age <= Max['adult'])
recommended_difficulty(Age, 'high', beginner) <= (Min['adult'] <= Age) & (Age <= Max['adult'])
recommended_difficulty(Age, 'high', intermediate) <= (Min['adult'] <= Age) & (Age <= Max['adult'])
recommended_difficulty(Age, 'moderate', beginner) <= (Min['adult'] <= Age) & (Age <= Max['adult'])
recommended_difficulty(Age, 'moderate', intermediate) <= (Min['adult'] <= Age) & (Age <= Max['adult'])

# Recommended Difficulties and MET Values for middle_aged people(35-49)
recommended_met(Age, Activity, 'low') <= (Min['middle_aged'] <= Age) & (Age <= Max['middle_aged']) & (Activity != 'high')
recommended_met(Age, 'high', 'low') <= (Min['middle_aged'] <= Age) & (Age <= Max['middle_aged'])
recommended_met(Age, 'high', 'moderate') <= (Min['middle_aged'] <= Age) & (Age <= Max['middle_aged'])

recommended_difficulty(Age, Activity, beginner) <= (Min['middle_aged'] <= Age) & (Age <= Max['middle_aged'])
recommended_difficulty(Age, 'moderate', intermediate) <= (Min['middle_aged'] <= Age) & (Age <= Max['middle_aged'])
recommended_difficulty(Age, 'high', intermediate) <= (Min['middle_aged'] <= Age) & (Age <= Max['middle_aged'])

# Recommended MET Values for Older_adults(50+)
recommended_met(Age, Activity, 'low') <= (Age >= Min['older_adult']) & (Activity != 'high')
recommended_met(Age, 'high', 'low') <= (Age >= Min['older_adult'])
recommended_met(Age, 'high', 'moderate') <= (Age >= Min['older_adult'])

#Only beginner Difficulty assigned to Older_adults
recommended_difficulty(Age, Activity, beginner) <= (Age >= Min['older_adult'])



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 10.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pydatalog: filename=pydatalog-0.17.4-cp311-cp311-linux_x86_64.whl size=1549120 sha256=92fa7d0d41f21b6d5fc0d0797f2d6ff8d0be5cbd0fa000c6ccc78a321fa21c7b
  Stored in directory: /root/.cache/pip/wheels/8f/f8/a0/c12514fe74e5f69d0d8967bd92363ea7a2961589ead1350b1d
Successfully built pydatalog


recommended_difficulty(Age,Activity,'beginner') <=

In [5]:
#Same as the filtering model of the previous files; it will just return a filtered_workouts.csv file filtered on the basis of user's intensity and activity level

#Cleaning the diffiuclty column for trailing whitespaces
df = pd.read_csv("workouts.csv")
df['Difficulty'] = df['Difficulty'].str.strip().str.capitalize()

# MET Ranges mapping
MET_RANGES = {
    'low': (1.5, 3.9),
    'moderate': (4.0, 6.9),
    'high': (7.0, 10.0)
}

def get_user_input():
    return {
        'age': 18,
        'activity': 'high'
    }

def get_recommendations(age, activity):
    met_output = recommended_met(age, activity, MetRange).data
    diff_output = recommended_difficulty(age, activity, Difficulty).data

    met_recs = [item[0] for item in met_output]
    diff_recs = [item[0].capitalize() for item in diff_output]

    return met_recs, diff_recs

def filter_workouts(df, met_recs, diff_recs):
    met_mask = pd.Series([False] * len(df))
    for met_level in met_recs:
        if met_level in MET_RANGES:
            min_val, max_val = MET_RANGES[met_level]
            met_mask |= df['MET Value'].between(min_val, max_val)

    filtered_df = df[met_mask & df['Difficulty'].isin(diff_recs)]
    return filtered_df


def main():
    met_recs, diff_recs = get_recommendations(USER_AGE, USER_ACTIVITY_LEVEL)

    print("Recommended MET Levels:", met_recs)
    print("Recommended Difficulties:", diff_recs)

    personalized_df = filter_workouts(df, met_recs, diff_recs)

    if not personalized_df.empty:
        print("\n Personalized Workout Recommendations:\n")
        print(personalized_df)
    else:
        print("\n No workouts match the given filters.")

    personalized_df.to_csv("filtered_workouts.csv", index=False)
main()


Recommended MET Levels: ['moderate', 'low']
Recommended Difficulties: ['Intermediate', 'Beginner']

 Personalized Workout Recommendations:

                              Name Target Muscle        Type  MET Value  \
0            Child's pose Stretch    Middle Back  Stretching        2.0   
1      Front and Back Neck Stretch   Middle Back  Stretching        2.0   
2             Lateral Neck Stretch   Middle Back  Stretching        2.0   
3                 Shoulder Circles    Quadriceps  Stretching        2.0   
4                    Ankle Circles        Calves    Mobility        2.3   
..                             ...           ...         ...        ...   
380           Lateral Bounding Hop    Quadriceps      Cardio        6.5   
381              Staggered push-up         Chest    Strength        6.5   
382                   Frog push-up         Chest    Strength        6.5   
383  Alternating Staggered Push-Up         Chest    Strength        6.5   
384                   Pike Push-Up 

In [6]:
#Generating a weekly plan with rest days
#!!It is not a populated plan just a blue print for the populated plan to work on!!

# constants
DAYS_OF_WEEK = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
GOAL_FOCUS_TEMPLATES = {
    'stay_fit': [
        'Upper Body Strength', 'Lower Body Strength', 'Core + Abs',
        'Light Endurance', 'Light Endurance'
    ],
    'gain_weight': [
        'Upper Body Strength', 'Core + Chest', 'Lower Body Strength',
        'Light Endurance', 'Full Body Strength'
    ],
    'weight_loss': [
        'Cardio', 'Full Body HIIT', 'Core + Lower Body',
        'Full Body HIIT', 'Abs + Upper Body'
    ],
    'build_muscle': [
        'Upper Body Strength', 'Lower Body Strength', 'Upper Body Strength',
        'Core + Abs', 'Cardio'
    ]
}

# Cleaners
def get_weekday(day_index):
    return DAYS_OF_WEEK[day_index % 7]

def get_next_focus(template_list, day_number):
    return template_list[day_number % len(template_list)]

#Main plan generation
def generate_plan(goal, program_duration, preferred_rest_day):
    plan = []
    focus_template = GOAL_FOCUS_TEMPLATES[goal]
    focus_day_counter = 0
    workout_streak = 0  # Count how many workout days since last rest

    for i in range(program_duration):
        weekday = get_weekday(i)

        if weekday == preferred_rest_day:
            focus = 'Complete Rest Day'
            workout_streak = 0  # reset streak after rest
        elif workout_streak >= 3:
            focus = 'Active Rest Day'
            workout_streak = 0
        else:
            focus = get_next_focus(focus_template, focus_day_counter)
            focus_day_counter += 1
            workout_streak += 1

        plan.append({
            'Day': f'Day {i+1} ({weekday})',
            'Focus': focus
        })

    return plan

def print_plan(plan):
    for entry in plan:
        print(entry)

def export_to_csv(plan, filename='weekly_plan_with_focus.csv'):
    pd.DataFrame(plan).to_csv(filename, index=False)

weekly_plan = generate_plan(USER_GOAL, USER_PROGRAM_DURATION, USER_REST_DAY)
print_plan(weekly_plan)
export_to_csv(weekly_plan)

{'Day': 'Day 1 (Monday)', 'Focus': 'Cardio'}
{'Day': 'Day 2 (Tuesday)', 'Focus': 'Full Body HIIT'}
{'Day': 'Day 3 (Wednesday)', 'Focus': 'Core + Lower Body'}
{'Day': 'Day 4 (Thursday)', 'Focus': 'Active Rest Day'}
{'Day': 'Day 5 (Friday)', 'Focus': 'Full Body HIIT'}
{'Day': 'Day 6 (Saturday)', 'Focus': 'Abs + Upper Body'}
{'Day': 'Day 7 (Sunday)', 'Focus': 'Complete Rest Day'}
{'Day': 'Day 8 (Monday)', 'Focus': 'Cardio'}
{'Day': 'Day 9 (Tuesday)', 'Focus': 'Full Body HIIT'}
{'Day': 'Day 10 (Wednesday)', 'Focus': 'Core + Lower Body'}
{'Day': 'Day 11 (Thursday)', 'Focus': 'Active Rest Day'}
{'Day': 'Day 12 (Friday)', 'Focus': 'Full Body HIIT'}
{'Day': 'Day 13 (Saturday)', 'Focus': 'Abs + Upper Body'}
{'Day': 'Day 14 (Sunday)', 'Focus': 'Complete Rest Day'}
{'Day': 'Day 15 (Monday)', 'Focus': 'Cardio'}
{'Day': 'Day 16 (Tuesday)', 'Focus': 'Full Body HIIT'}
{'Day': 'Day 17 (Wednesday)', 'Focus': 'Core + Lower Body'}
{'Day': 'Day 18 (Thursday)', 'Focus': 'Active Rest Day'}
{'Day': 'Day 19 (

In [7]:
#The actual population of the weekly plan with user specific workouts' number as well as duration and calories calculations
#It will return a 'final_populated_workouts' file in JSON format that will be the actual workout plan for a user


# Configuration of exercises, sets, and durations
workout_config = {
    'adult': {
        'low': {
            'weight_loss':     {'exercises': 3, 'sets': 2, 'duration_per_exercise': 4},
            'stay_fit':        {'exercises': 3, 'sets': 2, 'duration_per_exercise': 4},
            'build_muscle':    {'exercises': 4, 'sets': 3, 'duration_per_exercise': 5},
            'gain_weight':     {'exercises': 4, 'sets': 3, 'duration_per_exercise': 5},
        },
        'moderate': {
            'weight_loss':     {'exercises': 4, 'sets': 3, 'duration_per_exercise': 5},
            'stay_fit':        {'exercises': 4, 'sets': 3, 'duration_per_exercise': 5},
            'build_muscle':    {'exercises': 5, 'sets': 4, 'duration_per_exercise': 6},
            'gain_weight':     {'exercises': 5, 'sets': 4, 'duration_per_exercise': 6},
        },
        'high': {
            'weight_loss':     {'exercises': 5, 'sets': 3, 'duration_per_exercise': 6},
            'stay_fit':        {'exercises': 5, 'sets': 3, 'duration_per_exercise': 6},
            'build_muscle':    {'exercises': 6, 'sets': 4, 'duration_per_exercise': 7},
            'gain_weight':     {'exercises': 6, 'sets': 4, 'duration_per_exercise': 7},
        }
    },
    'middle_aged': {
        'low': {
            'weight_loss':     {'exercises': 3, 'sets': 2, 'duration_per_exercise': 4},
            'stay_fit':        {'exercises': 3, 'sets': 2, 'duration_per_exercise': 4},
            'build_muscle':    {'exercises': 3, 'sets': 3, 'duration_per_exercise': 5},
            'gain_weight':     {'exercises': 3, 'sets': 3, 'duration_per_exercise': 5},
        },
        'moderate': {
            'weight_loss':     {'exercises': 4, 'sets': 2, 'duration_per_exercise': 5},
            'stay_fit':        {'exercises': 4, 'sets': 2, 'duration_per_exercise': 5},
            'build_muscle':    {'exercises': 4, 'sets': 3, 'duration_per_exercise': 6},
            'gain_weight':     {'exercises': 4, 'sets': 3, 'duration_per_exercise': 6},
        },
        'high': {
            'weight_loss':     {'exercises': 5, 'sets': 3, 'duration_per_exercise': 5},
            'stay_fit':        {'exercises': 5, 'sets': 3, 'duration_per_exercise': 5},
            'build_muscle':    {'exercises': 5, 'sets': 3, 'duration_per_exercise': 6},
            'gain_weight':     {'exercises': 5, 'sets': 3, 'duration_per_exercise': 6},
        }
    },
    'older_adult': {
        'low': {
            'weight_loss':     {'exercises': 2, 'sets': 2, 'duration_per_exercise': 3},
            'stay_fit':        {'exercises': 2, 'sets': 2, 'duration_per_exercise': 3},
            'build_muscle':    {'exercises': 2, 'sets': 2, 'duration_per_exercise': 4},
            'gain_weight':     {'exercises': 2, 'sets': 2, 'duration_per_exercise': 4},
        },
        'moderate': {
            'weight_loss':     {'exercises': 3, 'sets': 2, 'duration_per_exercise': 4},
            'stay_fit':        {'exercises': 3, 'sets': 2, 'duration_per_exercise': 4},
            'build_muscle':    {'exercises': 3, 'sets': 2, 'duration_per_exercise': 5},
            'gain_weight':     {'exercises': 3, 'sets': 2, 'duration_per_exercise': 5},
        },
        'high': {
            'weight_loss':     {'exercises': 4, 'sets': 2, 'duration_per_exercise': 4},
            'stay_fit':        {'exercises': 4, 'sets': 2, 'duration_per_exercise': 4},
            'build_muscle':    {'exercises': 4, 'sets': 3, 'duration_per_exercise': 5},
            'gain_weight':     {'exercises': 4, 'sets': 3, 'duration_per_exercise': 5},
        }
    }
}

def calculate_calories_burned(met, duration_minutes, weight_kg):
    return (met * 3.5 * weight_kg / 200) * duration_minutes

# User-provided values (replace these with your dynamic inputs)
USER_AGE_GROUP = "adult"
USER_ACTIVITY_LEVEL = "moderate"
USER_GOAL = "weight_loss"
USER_WEIGHT = 65  # in kg

# Load workout and plan CSVs
workouts_df = pd.read_csv("filtered_workouts.csv")
plan_df = pd.read_csv("weekly_plan_with_focus.csv")

# Focus Area Definitions
focus_area_definitions = {
    'Upper Body Strength': {
        'target_muscles': ['Forearms', 'Shoulders', 'Biceps', 'Triceps', 'Chest'],
        'Type': ['Strength']
    },
    'Lower Body Strength': {
        'target_muscles': ['Hamstrings', 'Glutes', 'Quadriceps', 'Calves', 'Abductors', 'Adductors'],
        'Type': ['Strength']
    },
    'Core + Abs': {
        'target_muscles': ['Abdominals'],
        'Type': ['Strength', 'Core']
    },
    'Core + Chest': {
        'target_muscles': ['Abdominals', 'Chest'],
        'Type': ['Strength', 'Core']
    },
    'Light Endurance': {
        'target_muscles': ['Quadriceps', 'Calves', 'Glutes', 'Hamstrings'],
        'Type': ['Cardio', 'Mobility']
    },
    'Full Body Strength': {
        'target_muscles': ['Full Body', 'Biceps', 'Triceps', 'Shoulders', 'Glutes', 'Hamstrings'],
        'Type': ['Strength']
    },
    'Full Body HIIT': {
        'target_muscles': [],
        'Type': [],
        'Caution': ['HIIT', 'Plyometric HIIT', 'Isometric HIIT']
    },
    'Core + Lower Body': {
        'target_muscles': ['Abdominals','Hamstrings', 'Glutes', 'Quadriceps', 'Calves', 'Abductors', 'Adductors'],
        'Type': ['Strength', 'Core']
    },
    'Abs + Upper Body': {
        'target_muscles': ['Abdominals', 'Shoulders', 'Biceps', 'Triceps', 'Middle Back', 'Lower Back'],
        'Type': ['Strength']
    },
    'Cardio': {
        'target_muscles': [],
        'Type': ['Cardio']
    },
    'Active Rest Day': {
        'target_muscles': [],
        'Type': ['Mobility', 'Stretching']
    }
}

def smart_get_workouts_for_focus(focus_area, activity_level):
    fallback_focus_map = {
        'Core + Lower Body Strength': ['Lower Body Strength', 'Core + Abs', 'Lower Body Strength'],
        'Full Body HIIT': ['Light Endurance', 'Cardio'],
        'Core + Chest': ['Core + Abs', 'Upper Body Strength'],
        'Abs + Upper Body': ['Core + Abs', 'Upper Body Strength'],
    }

    # Define light fallback: mobility/stretch
    default_light_fallback = workouts_df[
        (workouts_df['Type'].isin(['Mobility', 'Stretching'])) &
        (workouts_df['Difficulty'] == 'Beginner')
    ]

    def filter_workouts(focus):
        filters = focus_area_definitions.get(focus, {})
        if not filters:
            return []

        # Prepare filters
        muscle_filter = workouts_df['Target Muscle'].isin(filters.get('target_muscles', [])) if filters.get('target_muscles') else True
        type_filter = workouts_df['Type'].isin(filters.get('Type', [])) if filters.get('Type') else True
        caution_filter = workouts_df['Caution'].isin(filters.get('Caution', [])) if filters.get('Caution') else True

        # Apply combination
        if isinstance(muscle_filter, bool):
            filtered = workouts_df[type_filter & caution_filter]
        elif isinstance(type_filter, bool):
            filtered = workouts_df[muscle_filter & caution_filter]
        elif isinstance(caution_filter, bool):
            filtered = workouts_df[muscle_filter & type_filter]
        else:
            filtered = workouts_df[muscle_filter & type_filter & caution_filter]

        # Special case: Light Endurance
        if focus == 'Light Endurance':
            filtered = filtered[
                (filtered['Difficulty'].isin(['Beginner', 'Intermediate'])) &
                (filtered['MET Value'] <= 6)
            ]

        return filtered

    # Step 1: Try strict match
    primary_workouts = filter_workouts(focus_area)
    if not primary_workouts.empty:
        return primary_workouts.sort_values(by='MET Value').to_dict('records')

    # Step 2: Try fallback focus areas
    fallback_focuses = fallback_focus_map.get(focus_area, [])
    for alt_focus in fallback_focuses:
        fallback_workouts = filter_workouts(alt_focus)
        if not fallback_workouts.empty:
            return fallback_workouts.sort_values(by='MET Value').to_dict('records')

    # Step 3: Relax difficulty (e.g. include Intermediate for Beginners)
    if activity_level.lower() == 'low':
        relaxed = workouts_df[
            (workouts_df['Difficulty'].isin(['Beginner', 'Intermediate'])) &
            (workouts_df['MET Value'] <= 6)
        ]
        if not relaxed.empty:
            return relaxed.sort_values(by='MET Value').to_dict('records')

    # Step 4: Fallback to Mobility / Stretching
    if not default_light_fallback.empty:
        return default_light_fallback.sort_values(by='MET Value').to_dict('records')

    # Step 5: Nothing found
    return []


# Build final plan
age_group = USER_AGE_GROUP
activity_level = USER_ACTIVITY_LEVEL
goal = USER_GOAL
weight = USER_WEIGHT

config = workout_config[age_group][activity_level][goal]
num_exercises = config['exercises']
sets = config['sets']
duration_per_ex = config['duration_per_exercise']

focus_index_tracker = {}
final_plan = []

for _, row in plan_df.iterrows():
    day_data = dict(row)
    focus = row['Focus']
    day_data['Workouts'] = []
    total_duration = 0
    total_calories = 0

    if 'Rest' in focus:
        day_data['Total Duration (min)'] = 0
        day_data['Total Calories Burned'] = 0
    else:
        pool = smart_get_workouts_for_focus(focus, activity_level)
        start = focus_index_tracker.get(focus, 0)
        end = start + num_exercises
        selected = pool[start:end]

        if len(selected) < num_exercises:
            selected += pool[:num_exercises - len(selected)]
        focus_index_tracker[focus] = end % len(pool)

        for w in selected:
            met = w.get('MET Value', 3)
            duration = sets * duration_per_ex
            calories = calculate_calories_burned(met, duration, weight)

            w['Sets'] = sets
            w['Duration (min)'] = duration
            w['Calories Burned'] = round(calories, 2)

            total_duration += duration
            total_calories += calories
            day_data['Workouts'].append(w)

        day_data['Total Duration'] = total_duration
        day_data['Total Calories Burned'] = round(total_calories, 2)

    final_plan.append(day_data)

# Save final plan to JSON
with open("final_populated_workout_plan.json", "w") as f:
    json.dump(final_plan, f, indent=2)

print("Final structured workout plan saved to 'final_populated_workout_plan.json'")


Final structured workout plan saved to 'final_populated_workout_plan.json'
